In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

from skimage.transform import hough_circle, hough_circle_peaks, hough_ellipse
from skimage.feature import canny
from skimage.morphology import remove_small_objects
from skimage.draw import circle_perimeter, circle
from skimage.util import img_as_ubyte
from pathlib import Path
from skimage import data, color
plt.rcParams["figure.figsize"] = (20, 10)
import torch
from narsil.segmentation.network import basicUnet, smallerUnet
from narsil.liverun.utils import padTo32
import math
from datetime import datetime
import time
from scipy.signal import find_peaks
from skimage.filters import gaussian

In [3]:

#imgpath = Path('/home/pk/Documents/realtimeData/hetero40x/Pos103/phaseFast/img_000000017.tiff')


#imgpath = Path('/home/pk/Documents/EXP-21-BY1006/therun/Pos11/phase/img_000000008.tiff')

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

#cellSegModelPath = Path('/home/pk/Documents/models/mixed10epochs_betterscale_contrastAdjusted1.pth')

modelPath = Path('/home/pk/Documents/models/channels.pth')

pad = padTo32()

with torch.no_grad():
    cellNetState = torch.load(modelPath)
    
    if cellNetState['modelParameters']['netType'] == 'big':
        cellSegNet = basicUnet(cellNetState['modelParameters']['transposeConv'])
    elif cellNetState['modelParameters']['netType'] == 'small':
        cellSegNet = smallerUnet(cellNetState['modelParameters']['transposeConv'])
        
    cellSegNet.load_state_dict(cellNetState['model_state_dict'])
    cellSegNet.to(device)
    cellSegNet.eval()

In [4]:
def imgFilenameFromNumber(number):
    if number == 0:
        num_digits = 1
    else:
        num_digits = int(math.log10(number)) + 1
    imgFilename = 'img_' + '0' * (9 - num_digits) + str(number) + '.tiff'
    return imgFilename


In [5]:
def barcodesFromPeaks(one_img, minLengthOfChannel= 200,
                      minPeaksDistance=25, gapWidth=48,
                      numChannels=21):
    
    hist = np.sum(one_img, axis = 0) > minLengthOfChannel

    peaks, _ = find_peaks(hist, distance=minPeaksDistance, plateau_size=15)
    
    indices_with_larger_gaps = np.where(np.ediff1d(peaks) > gapWidth)[0]
    
    locations_before_barcode = peaks[indices_with_larger_gaps]
    locations_after_barcode = peaks[indices_with_larger_gaps + 1]
    
    locations_barcode = np.rint(np.mean((locations_before_barcode,
                                        locations_after_barcode), axis = 0)).astype('int')
    
    
    num_barcodes = len(locations_barcode)
    # there are 5 barcodes seen in the image
    if num_barcodes == 5:
        # count the number of channels before the first barcode and after the 
        # last barcode and include them upto numChannels channels
        y_channels = []
        
        # channels before first barcode
        indices_before_first = np.where(peaks < locations_barcode[0])[0]
        y_channels.extend(list(peaks[indices_before_first]))
        
        for i in range(num_barcodes):
            indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                             peaks < locations_barcode[i]))[0]
            y_channels.extend(list(peaks[indices]))
            
        # number of channels to count after the last
        number_to_include = numChannels - len(indices_before_first)
        indices_after_last = np.where(peaks > locations_barcode[-1])[0]
        y_channels.extend(list(peaks[indices_after_last][:number_to_include]))
        
    elif num_barcodes == 6:
        y_channels = []
        # count only the channels between barcodes and 
        # grab the (x, y) locations to cut,
        # x will be the top of the channel, row number
        # y will be the peak picked up in the histogram, between the barcodes
        # count 21 channels after calculating
        for i in range(num_barcodes):
            indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                             peaks < locations_barcode[i]))[0]
            #if len(indices) == 21:
            # all good pick them up
            y_channels.extend(list(peaks[indices]))   
        
    else:
        # detection failure, since it is ambiguous skipp the position
        y_channels = []
        print(f"Detection failure, {num_barcodes} detected")
    
    # locations of the barcode and locations of channels to cut.
    return locations_barcode, y_channels

In [6]:
def apply(dirname, minLengthOfChannel = 200, minPeaksDistance = 25, gapWidth=48):
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    modelPath = Path('/home/pk/Documents/models/channels.pth')
    pad = padTo32()

    with torch.no_grad():
        cellNetState = torch.load(modelPath)

        if cellNetState['modelParameters']['netType'] == 'big':
            cellSegNet = basicUnet(cellNetState['modelParameters']['transposeConv'])
        elif cellNetState['modelParameters']['netType'] == 'small':
            cellSegNet = smallerUnet(cellNetState['modelParameters']['transposeConv'])

        cellSegNet.load_state_dict(cellNetState['model_state_dict'])
        cellSegNet.to(device)
        cellSegNet.eval()
    start = time.time()
    count = 0
    for i in range(102, 841):
        imgfilename = dirname + "Pos" + str(i) + '/phaseFast/img_000000000.tiff' 
        imgpath = Path(imgfilename)
        image = io.imread(imgpath)
        image = pad(image)
        imgTensor = torch.from_numpy(image.astype('float32')).unsqueeze(0).unsqueeze(0).to(device)
        imgTensor = (imgTensor - torch.mean(imgTensor))/torch.std(imgTensor)
        out = torch.sigmoid(cellSegNet(imgTensor))
        out_cpu = out.detach().cpu().numpy().squeeze(0).squeeze(0)
        #print(imgTensor.shape)
        out_cpu = gaussian(out_cpu, sigma = 3)

        out_cpu = out_cpu > 0.5
        out_cpu = remove_small_objects(out_cpu.astype('bool'), min_size=100)
        locations_barcode, locations_channels = barcodesFromPeaks(out_cpu)
        print(f"Barcodes: {len(locations_barcode)} --- no channels: {len(locations_channels)}")
        
        if len(locations_channels) != 105:
            count += 1
        #plt.figure()
        #plt.imshow(out_cpu)
        #plt.plot(hist)
        #plt.plot(peaks, hist[peaks], 'r*')
        #plt.plot(peaks[indices_with_larger_gaps],
        #       hist[peaks[indices_with_larger_gaps]],'o', markersize=10,
        #         scalex=False, scaley=False, fillstyle='none')
        #plt.show()
        print(imgpath)
    duration = time.time() - start
        
    print(f"Duration: {duration/i}s")
    print(f"Failed count: {count}")
    del cellSegNet
    del cellNetState
    torch.cuda.empty_cache()
    return None

In [7]:
#one_img = apply('/home/pk/Documents/realtimeData/hetero40x/Pos103/phaseFast/')

In [8]:
#one_img = apply("/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/dry run/")

In [9]:
one_img = apply("/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/")

Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos102/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos103/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos104/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos105/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos106/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos107/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos108/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos163/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos164/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos165/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos166/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos167/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos168/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos169/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos225/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos226/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos227/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos228/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos229/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos230/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos231/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos287/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos288/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos289/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos290/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos291/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos292/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos293/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos348/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos349/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 83
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos350/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 83
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos351/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos352/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 100
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos353/phaseFast/img_000000000.tiff
Detection failure, 7 detected
Barcodes: 7 --- no channels: 0
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos354/phaseFast/img_000000000.tiff
Detection failure, 8 detected
Barcod

...

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos592/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos593/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos594/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos595/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos596/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos597/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos598/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos654/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos655/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos656/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos657/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos658/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos659/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos660/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos716/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos717/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos718/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos719/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos720/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos721/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos722/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos777/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos778/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos779/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos780/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos781/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos782/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos783/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0

Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos838/phaseFast/img_000000000.tiff
Barcodes: 6 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos839/phaseFast/img_000000000.tiff
Barcodes: 5 --- no channels: 105
/mnt/sda1/Praneeth/EXP-20-BP0361 75k imaging 6ugml/after loading/Pos840/phaseFast/img_000000000.tiff
Duration: 0.3186885918889727s
Failed count: 40


In [10]:
def barcodesFromPeaks(one_img, minLengthOfChannel= 200,
                      minPeaksDistance=25, gapWidth=48,
                      numChannels=21):
    
    hist = np.sum(one_img, axis = 0) > minLengthOfChannel

    peaks, _ = find_peaks(hist, distance=minPeaksDistance, plateau_size=15)
    
    indices_with_larger_gaps = np.where(np.ediff1d(peaks) > gapWidth)[0]
    
    locations_before_barcode = peaks[indices_with_larger_gaps]
    locations_after_barcode = peaks[indices_with_larger_gaps + 1]
    
    locations_barcode = np.rint(np.mean((locations_before_barcode,
                                        locations_after_barcode), axis = 0)).astype('int')
    
    
    num_barcodes = len(locations_barcode)
    # there are 5 barcodes seen in the image
    if num_barcodes == 5:
        # count the number of channels before the first barcode and after the 
        # last barcode and include them upto numChannels channels
        y_channels = []
        
        # channels before first barcode
        indices_before_first = np.where(peaks < locations_barcode[0])[0]
        y_channels.extend(list(peaks[indices_before_first]))
        
        for i in range(num_barcodes):
            indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                             peaks < locations_barcode[i]))[0]
            y_channels.extend(list(peaks[indices]))
            
        # number of channels to count after the last
        number_to_include = numChannels - len(indices_before_first)
        indices_after_last = np.where(peaks > locations_barcode[-1])[0]
        y_channels.extend(list(peaks[indices_after_last][:number_to_include]))
        
    elif num_barcodes == 6:
        y_channels = []
        # count only the channels between barcodes and 
        # grab the (x, y) locations to cut,
        # x will be the top of the channel, row number
        # y will be the peak picked up in the histogram, between the barcodes
        # count 21 channels after calculating
        for i in range(num_barcodes):
            indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                             peaks < locations_barcode[i]))[0]
            #if len(indices) == 21:
            # all good pick them up
            y_channels.extend(list(peaks[indices]))   
        
    else:
        # detection failure, since it is ambiguous skipp the position
        print("Detection failure")
    
    # locations of the barcode and locations of channels to cut.
    return locations_barcode, y_channels

In [10]:
barcodes, y_channels = barcodesFromPeaks(one_img)

In [12]:
barcodes

array([ 434, 1204, 1968, 2732, 3500])

In [13]:
len(y_channels)

105

In [8]:
#one_img = apply("/home/pk/Documents/EXP-21-BY1006/therun/")

In [12]:
plt.imshow(one_img)

In [30]:
minLengthOfChannel = 200

In [31]:
hist = np.sum(one_img, axis = 0) > minLengthOfChannel

In [32]:
plt.plot(hist)

In [24]:
plt.figure()
plt.imshow(one_img)
plt.show()

In [18]:
b = np.ones((10,))

In [19]:
b

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [24]:
np.insert(b, 0, 12)

array([12.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [ ]:
3.29

In [8]:
image = io.imread(imgpath)
image = pad(image)
imgTensor = torch.from_numpy(image.astype('float32')).unsqueeze(0).unsqueeze(0).to(device)
imgTensor = (imgTensor - torch.mean(imgTensor))/torch.std(imgTensor)
out = torch.sigmoid(cellSegNet(imgTensor))
out_cpu = out.detach().cpu().numpy().squeeze(0).squeeze(0)

In [9]:
out_cpu

array([[0.05213241, 0.01452362, 0.00998267, ..., 0.01133891, 0.01069751,
        0.0445163 ],
       [0.01521315, 0.00218285, 0.00127628, ..., 0.00178523, 0.00149088,
        0.01218481],
       [0.01415603, 0.00190436, 0.00120316, ..., 0.00213419, 0.00174523,
        0.01270408],
       ...,
       [0.00372532, 0.00029129, 0.00017153, ..., 0.00029864, 0.00035982,
        0.0064674 ],
       [0.0054258 , 0.00055616, 0.00038846, ..., 0.00059395, 0.00071317,
        0.00908298],
       [0.02848835, 0.00667637, 0.00519472, ..., 0.00587905, 0.00676154,
        0.03456329]], dtype=float32)

In [36]:
plt.imshow(out_cpu)

In [11]:
from skimage.measure import regionprops, label
from datetime import datetime

In [12]:
print(datetime.now())
props = regionprops(label(out_cpu > 0.9))
print(datetime.now())

2021-11-18 13:43:32.450126
2021-11-18 13:43:32.494567


In [35]:
removed_labels = []
labeled_img = label(out_cpu > 0.5)
plt.imshow(labeled_img, cmap='gnuplot2')

In [4]:
image = io.imread(path).astype('float32')

In [5]:
image.shape

(1488, 4096)

In [6]:
image.dtype

dtype('float32')

In [7]:
plt.imshow(image)

In [8]:
image = (image - np.mean(image))/np.std(image)

In [9]:
from skimage import filters

In [10]:
edges = canny(image, sigma=7)

In [11]:
plt.imshow(edges)

In [12]:
from scipy.ndimage import binary_fill_holes, binary_erosion

In [13]:
image = binary_fill_holes(edges)

In [15]:
plt.imshow(image)

In [16]:
image = binary_erosion(image)

In [17]:
plt.imshow(image)

In [31]:
def detect_circles(in_img):
    edges = canny(in_img, sigma=2)
    hough_radii = np.arange(10, , 2)
    hough_res = hough_circle(edges, hough_radii)
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=300)

    img1 = np.zeros(in_img.shape)
    img1 = color.gray2rgb(img1)
    for center_y, center_x, radius, (r, g, b, _) in zip(cy, cx, radii, 
                                          plt.cm.nipy_spectral(np.linspace(0,1, len(radii))) # color map
                                         ):
        circy, circx = circle(center_y, center_x, radius)
        img1[circy, circx] = (r*255, g*255, b*255)
    return img1


SyntaxError: invalid syntax (2565744971.py, line 3)

In [25]:
img = detect_circles(image)

/tmp/ipykernel_431803/2842679918.py:12: FutureWarning: `draw.circle` is deprecated in favor of `draw.disk`.`draw.circle` will be removed in version 0.19
  circy, circx = circle(center_y, center_x, radius)


In [27]:
plt.imshow(img)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
radii

In [ ]:
cx

In [ ]:

cy